In [4]:
# this needs to be run for each new runtime
# because colab has scikit-learn 1.0.2 pre-installed 
# and we need newer version (1.2.0 and higher)
# to use .set_output() method
#!pip install scikit-learn --upgrade

# if you plan on running the whole notebook again during the same runtime
# you can comment the line above

# Housing classification iter 4 Categorical encoding

So far, we have been only using numerical features for our model. By not using the categorical features, we were missing out on a lot of potentially important information.

As we will see, converting categorical features to numerical (so that they can be "digested" by the Scikit-Learn transformers and models) adds a bit of complexity to the modelling pipeline. This is why in this notebook we will start by encoding them without using pipelines (just to understand what's going) and only later we will include categorical encoding inside the pipeline.

Before going through this notebook, read the Platform lesson on One-Hot Encoding: https://platform.wbscodingschool.com/courses/data-science/12675/ 

## 1. Data reading & splitting

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline


# reading
house_iter6 = pd.read_csv(r'housing-classification-iter6.csv')

In [6]:
house_iter6

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,0,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,1,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,0,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [7]:
house_iter6['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', nan, 'Fa'], dtype=object)

Let's assume that cabins with the same cabin letter can be put into same 'category'

In [8]:
# # Feature Engineering
# data.loc[:, "Cabin"] = data.Cabin.str[0]
# data['Cabin'].unique()

In [27]:
# X and y creation
X = house_iter6.drop(columns=[ "CentralAir","MSZoning","Condition1","Street"])
y = X.pop("Expensive")

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

## 2. Categorical encoding - "MANUAL" approach  (Without using Pipelines)

### 2.1. Replacing NaNs

We will need two different strategies to deal with missing values in numerical and categorical features.

#### 2.1.1. Replacing NaNs in categorical features

We were imputing the mean to NaN’s on our preprocessing pipeline for numerical features. There's a problem with categorical values: they don’t have a “mean”. Here, we will replace NaNs with a string that marks them: “N_A”. It is not an elegant solution, but it will allow us to move forward.

In [10]:
# selecting non-numerical columns
X_train_cat = X_train.select_dtypes(exclude="number")

# defining the imputer to use "N_A" as replacement value
cat_imputer = SimpleImputer(strategy="constant", 
                            fill_value="N_A").set_output(transform='pandas')

# fitting and transforming
X_cat_imputed = cat_imputer.fit_transform(X_train_cat)

X_cat_imputed.head()

,Heating,Foundation,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,KitchenQual,FireplaceQu,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
318,GasA,PConc,Gd,TA,Gd,TA,Gd,GLQ,Gd,TA,...,Attchd,RFn,TA,TA,Y,N_A,N_A,N_A,WD,Normal
580,GasA,CBlock,TA,TA,TA,TA,No,BLQ,Gd,Gd,...,Attchd,Unf,TA,TA,Y,N_A,N_A,N_A,WD,Normal
961,GasA,CBlock,TA,Gd,Gd,Gd,No,ALQ,TA,TA,...,Attchd,Fin,TA,TA,Y,N_A,N_A,N_A,WD,Normal
78,GasA,CBlock,TA,TA,TA,TA,No,Unf,TA,N_A,...,N_A,N_A,N_A,N_A,Y,N_A,N_A,N_A,WD,Normal
5,GasA,Wood,TA,TA,Gd,TA,No,GLQ,TA,N_A,...,Attchd,Unf,TA,TA,Y,N_A,MnPrv,Shed,WD,Normal


#### 2.1.2. Replacing NaNs in numerical features

This is what we already did in previous notebooks: replacing numerical NaNs with the mean of their column.

In [32]:
# Selecting numerical columns
X_train_num = X_train.select_dtypes(include="number")

# Imputing the mean
num_imputer = SimpleImputer(strategy="mean").set_output(transform='pandas')

# Fitting and transforming
X_num_imputed = num_imputer.fit_transform(X_train_num)

X_num_imputed.head()

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,MSSubClass,...,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id
318,9900.0,90.00000,1347.0,4.0,1.0,0.0,3.0,340.0,0.0,60.0,...,9.0,1993.000000,656.0,60.0,144.0,0.0,0.0,4.0,2009.0,319.0
580,14585.0,69.58427,1144.0,3.0,2.0,0.0,2.0,216.0,0.0,20.0,...,7.0,1960.000000,572.0,110.0,0.0,0.0,0.0,6.0,2007.0,581.0
961,12227.0,69.58427,1330.0,4.0,1.0,0.0,2.0,550.0,0.0,60.0,...,11.0,1977.000000,619.0,282.0,0.0,0.0,0.0,7.0,2008.0,962.0
78,10778.0,72.00000,1768.0,4.0,0.0,0.0,0.0,0.0,0.0,90.0,...,8.0,1978.213442,0.0,0.0,0.0,0.0,0.0,4.0,2010.0,79.0
5,14115.0,85.00000,796.0,1.0,0.0,0.0,2.0,40.0,0.0,50.0,...,5.0,1993.000000,480.0,30.0,0.0,320.0,700.0,10.0,2009.0,6.0


In [12]:
# Concatenating all columns
X_imputed = pd.concat([X_cat_imputed, X_num_imputed], axis=1)

X_imputed.head()

,Heating,Foundation,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,KitchenQual,FireplaceQu,...,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id
318,GasA,PConc,Gd,TA,Gd,TA,Gd,GLQ,Gd,TA,...,9.0,1993.000000,656.0,60.0,144.0,0.0,0.0,4.0,2009.0,319.0
580,GasA,CBlock,TA,TA,TA,TA,No,BLQ,Gd,Gd,...,7.0,1960.000000,572.0,110.0,0.0,0.0,0.0,6.0,2007.0,581.0
961,GasA,CBlock,TA,Gd,Gd,Gd,No,ALQ,TA,TA,...,11.0,1977.000000,619.0,282.0,0.0,0.0,0.0,7.0,2008.0,962.0
78,GasA,CBlock,TA,TA,TA,TA,No,Unf,TA,N_A,...,8.0,1978.213442,0.0,0.0,0.0,0.0,0.0,4.0,2010.0,79.0
5,GasA,Wood,TA,TA,Gd,TA,No,GLQ,TA,N_A,...,5.0,1993.000000,480.0,30.0,0.0,320.0,700.0,10.0,2009.0,6.0


In [33]:
X_imputed_test = pd.concat([X_cat_imputed_test, X_num_imputed_test], axis=1)

X_imputed_test.head()

,Heating,Foundation,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,KitchenQual,FireplaceQu,...,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id
318,GasA,PConc,Gd,TA,Gd,TA,Gd,GLQ,Gd,TA,...,9.0,1993.000000,656.0,60.0,144.0,0.0,0.0,4.0,2009.0,319.0
580,GasA,CBlock,TA,TA,TA,TA,No,BLQ,Gd,Gd,...,7.0,1960.000000,572.0,110.0,0.0,0.0,0.0,6.0,2007.0,581.0
961,GasA,CBlock,TA,Gd,Gd,Gd,No,ALQ,TA,TA,...,11.0,1977.000000,619.0,282.0,0.0,0.0,0.0,7.0,2008.0,962.0
78,GasA,CBlock,TA,TA,TA,TA,No,Unf,TA,N_A,...,8.0,1978.213442,0.0,0.0,0.0,0.0,0.0,4.0,2010.0,79.0
5,GasA,Wood,TA,TA,Gd,TA,No,GLQ,TA,N_A,...,5.0,1993.000000,480.0,30.0,0.0,320.0,700.0,10.0,2009.0,6.0


### 2.2. One Hot encoding

As you have learnt in the Platform lesson, One Hot encoding means creating a new binary column for each category in every categorical column. Fortunately, a Scikit-Learn transformer takes care of everything.

#### 2.3.1. Fitting the `OneHotEncoder`

As with any transformer, we have to:
1. Import it
2. Initialize it
3. Fit it to the data
4. Use it to transform the data

In [13]:
# import
from sklearn.preprocessing import OneHotEncoder

# initialize
my_onehot = OneHotEncoder(drop="first",sparse_output=False).set_output(transform='pandas')

# fit
my_onehot.fit(X_cat_imputed)

# transform
X_cat_imputed_onehot = my_onehot.transform(X_cat_imputed)

NOTE: If we leave `sparse_output=True`, the result will be a "sparse matrix": an object that Scikit-Learn creates when a matrix contains mostly zeros. In that case we would not be able to use `.set_output(transform='pandas')`.

In [14]:
X_cat_imputed_onehot

,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
318,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
580,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
961,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
78,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1122,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1346,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1406,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


We can see that all the columns contain either 0's or 1's. This is exactly how "one-hot" encoded columns (also called "dummy columns") look like: binary categories.

Each column has the name of the original column, an underscore and the name of the category: 

- A column such as "Sex", with only two categories, "male" and "female", has become a single column, "Sex_male", where `1` stands for "male" and `0` for "female".

- A column such as "basement_quality", with many categories ("A", "B", "C", "D", "E", "F", "G", "N_A", "T") has become as many columns as categories were (minus one), making the dataframe very wide and sparse.

### 2.3.3. Concatenating "one-hot" columns with numerical columns:

Now that the categorical columns are numerical, we can join them back with the originally numerical columns and assemble the dataset that will be ready for modelling:

In [15]:
X_imputed = pd.concat([X_cat_imputed_onehot, X_num_imputed], axis=1)

X_imputed

,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,...,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id
318,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,9.0,1993.000000,656.0,60.0,144.0,0.0,0.0,4.0,2009.0,319.0
580,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7.0,1960.000000,572.0,110.0,0.0,0.0,0.0,6.0,2007.0,581.0
961,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,11.0,1977.000000,619.0,282.0,0.0,0.0,0.0,7.0,2008.0,962.0
78,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,8.0,1978.213442,0.0,0.0,0.0,0.0,0.0,4.0,2010.0,79.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,5.0,1993.000000,480.0,30.0,0.0,320.0,700.0,10.0,2009.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7.0,1966.000000,484.0,40.0,0.0,0.0,0.0,7.0,2008.0,1042.0
1122,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,5.0,1956.000000,288.0,0.0,0.0,0.0,0.0,10.0,2009.0,1123.0
1346,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,9.0,1968.000000,508.0,80.0,0.0,290.0,0.0,6.0,2006.0,1347.0
1406,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,5.0,1988.000000,396.0,0.0,0.0,0.0,0.0,3.0,2009.0,1407.0


## 3. Categorical encoding - "Automated" approach (Using Pipelines)

In the manual approach, to encode the categorical columns numericall, we have:

1. Selected the categorical columns.
2. Fitted a `OneHotEncoder` to them.
3. Transformed the categorical columns with the encoder.
4. Concatenated the one-hot columns with the numerical columns.

All these steps can be synthetised by using Scikit-Learn Pipelines and specifically something called `ColumnTransformer`, which allows us to apply different transformations to two or more groups of columns: in our case, categorical and numerical columns.

To select numerical and categorical columns we can use `make_column_seletor` directly when constructing a transformer

This process is also called creating "branches" in the pipeline. One branch for the categorical columns and another for the numerical columns. Each branch will contain as many transformers as we want. Then, the branches will meet again, and the transformed columns will be automatically concatenated. Let's see the process in action:

### 3.1. Creating the "numeric pipe" and the "categoric pipe"

In [16]:
# create numerical pipeline, only with the SimpleImputer(strategy="mean")
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))
 
# create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(sparse_output=False)
)

### 3.2. Using `make_column_transformer` on 2 branches/pipelines (the `preprocessor`) 

We simply tell the pipeline the following:

- One branch, called `"numeric_pipe"`, will apply the steps in the `numeric_pipe` to the columns that `make_column_selector` selects (`(dtype_include='number')`)
- The second branch, called `"categoric_pipe"`, will apply the steps in the `categoric_pipe` to the columns `make_column_selector` selects (`(dtype_include='object')`)

`make_column_selector` example:

In [17]:
from sklearn.compose import make_column_selector
make_column_selector(dtype_include='number')(X_train)

['LotArea',
 'LotFrontage',
 'TotalBsmtSF',
 'BedroomAbvGr',
 'Fireplaces',
 'PoolArea',
 'GarageCars',
 'WoodDeckSF',
 'ScreenPorch',
 'MSSubClass',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'GarageYrBlt',
 'GarageArea',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'MiscVal',
 'MoSold',
 'YrSold',
 'Id']

In [18]:
X_cat_columns = X_train.select_dtypes(include="number").copy().columns
X_cat_columns

Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'KitchenAbvGr', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'MiscVal', 'MoSold',
       'YrSold', 'Id'],
      dtype='object')

In [19]:
from sklearn.compose import make_column_transformer


preprocessor = make_column_transformer(
        (numeric_pipe, make_column_selector(dtype_include='number')),
        (categoric_pipe, make_column_selector(dtype_include='object'))
)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDB62DC0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='N_A',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDAC6A00>)])

Similar to `Pipeline` and `make_pipeline`, instead of `make_column_transformer` we could have used `ColumnTransformer` to construct our transformer but in that case, we would have needed to name the steps ourself

In [20]:
from sklearn.compose import ColumnTransformer

preprocessor1 = ColumnTransformer([
        ('numeric_pipe',numeric_pipe, make_column_selector(dtype_include='number')),
        ('categoric_pipe',categoric_pipe, make_column_selector(dtype_include='object'))]
)
preprocessor1

ColumnTransformer(transformers=[('numeric_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDF589A0>),
                                ('categoric_pipe',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='N_A',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDF580D0>)])

### 3.3. Creating the `full_pipeline` (`preprocessor` + Decision Tree)

Pipelines are modular. The `preprocessor` we created above with the `ColumnTransformer` can become now a step in a new pipeline, that we'll call `full_piepline` and will include, as a last step, a Decision Tree model:

In [21]:
full_pipeline = make_pipeline(preprocessor, 
                              DecisionTreeClassifier()).set_output(transform='pandas')

In [22]:
full_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDB62DC0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N_A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDAC6A00>)])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

We can then fit this `full_pipeline` to the data:

Note: notice that we did not fit the `preprocessor` before —we only fit the pipeline once it has been full assembled.

In [28]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDB62DC0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N_A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDAC6A00>)])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

This full pipeline can make predictions, as any othet pipeline that ends with a model:

In [24]:
#full_pipeline.predict(X_train)

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [29]:
# Train
y_train_predict = full_pipeline.predict(X_train)
accuracy_score(y_train, y_train_predict)

1.0

### 3.4 Accessing steps of the pipeline

The estimators of a pipeline are stored as a list in the steps attribute, but can be accessed by index or name by indexing the Pipeline:

In [35]:
# first step in our pipeline is ColumnTransformer

# we can access it with an index
full_pipeline[0]

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDB62DC0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='N_A',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDAC6A00>)])

In [36]:
# or by it's name
full_pipeline['columntransformer']

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDB62DC0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='N_A',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DDAC6A00>)])

`named_steps` helps you navigate through the pipeline.   
When using it, you get the autocompletion active after every step.  
With `get_features_names_out()' you can get printed column names that specific estimator used

In [37]:
(
full_pipeline                          # Start with the full_pipeline
 .named_steps.columntransformer        # .named_steps will call “coloumntransformer” which is a ColumnTransformer
 .named_transformers_['pipeline-1']    # .named_transformer_ will call['pipeline-1'] 'pipeline-1' which is a first pipeline inside “coloumntransformer
 .named_steps.simpleimputer            # The second .named_steps will call “simpleimputer” which is the desired step.
 .get_feature_names_out()              # .get_feature_names_out() will get us the fetures that got into that desired step
)

array(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr',
       'Fireplaces', 'PoolArea', 'GarageCars', 'WoodDeckSF',
       'ScreenPorch', 'MSSubClass', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'KitchenAbvGr', 'TotRmsAbvGrd', 'GarageYrBlt',
       'GarageArea', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'MiscVal', 'MoSold', 'YrSold', 'Id'], dtype=object)

We can also get parameters out of any specific step e.g. catogeries from onehotencoder step:

In [38]:
full_pipeline.named_steps.columntransformer.named_transformers_['pipeline-2'].named_steps.onehotencoder.categories_

[array(['Floor', 'GasA', 'GasW', 'Grav', 'OthW', 'Wall'], dtype=object),
 array(['BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'Po', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'N_A', 'TA'], dtype=object),
 array(['Fa', 'Gd', 'N_A', 'Po', 'TA'], dtype=object),
 array(['Av', 'Gd', 'Mn', 'N_A', 'No'], dtype=object),
 array(['ALQ', 'BLQ', 'GLQ', 'LwQ', 'N_A', 'Rec', 'Unf'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'N_A', 'Po', 'TA'], dtype=object),
 array(['Grvl', 'N_A', 'Pave'], dtype=object),
 array(['IR1', 'IR2', 'IR3', 'Reg'], dtype=object),
 array(['Bnk', 'HLS', 'Low', 'Lvl'], dtype=object),
 array(['AllPub', 'NoSeWa'], dtype=object),
 array(['Corner', 'CulDSac', 'FR2', 'FR3', 'Inside'], dtype=object),
 array(['Gtl', 'Mod', 'Sev'], dtype=object),
 array(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
        'Crawfor',

### **Exercise 1:** use the new Pipeline with branches to train a DecisionTree with GridSearch cross validation.

We are basically asking to combine what you have learned in this notebook (categorical encoding & branches) with what you learned in the previous one (using `GridSearchCV` for a whole Pipeline).

In [39]:
# your code here

# solution:

numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))
 
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(sparse_output=False, handle_unknown='ignore') # handle_unknown is set to ignore the values that are not familiar
)

preprocessor = make_column_transformer(
        (numeric_pipe, make_column_selector(dtype_include='number')),
        (categoric_pipe, make_column_selector(dtype_include='object'))
)
preprocessor



from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor, 
                              DecisionTreeClassifier()).set_output(transform='pandas')

param_grid = {
    "columntransformer__pipeline-1__simpleimputer__strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14, 2),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2)
}

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DE01A340>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                                         (...
                                                                                                        sparse_output=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DE01A3D0>)])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'columntransformer__pipeline-1__simpleimputer__strategy': ['mean',
                                                                                    'median'],
                         'decisiontreeclassifier__max_depth': range(2, 14, 2),
                         'decisiontreeclassifier__min_samples_leaf': range(3, 12, 2)},
             verbose=1)

In [40]:
search.best_params_

{'columntransformer__pipeline-1__simpleimputer__strategy': 'mean',
 'decisiontreeclassifier__max_depth': 6,
 'decisiontreeclassifier__min_samples_leaf': 7}

In [41]:
search.best_score_

0.9272147023219984

In [42]:
# training accuracy
y_train_pred_1 = search.predict(X_train)

accuracy_score(y_train, y_train_pred_1)

0.9657534246575342

In [ ]:
# y_test_pred_1 = search.predict(X_test)

# accuracy_score(y_test, y_test_pred_1)

In [43]:
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))
 
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist', min_frequency=6) # handle_unknown is set to ignore the values that are not familiar
)

preprocessor = make_column_transformer(
        (numeric_pipe, make_column_selector(dtype_include='number')),
        (categoric_pipe, make_column_selector(dtype_include='object'))
)
preprocessor



from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor, 
                              DecisionTreeClassifier()).set_output(transform='pandas')

param_grid = {
    "columntransformer__pipeline-1__simpleimputer__strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14, 2),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2)
}

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DF035AC0>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                                         (...
                                                                                                        sparse_output=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DF035790>)])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'columntransformer__pipeline-1__simpleimputer__strategy': ['mean',
                                                                                    'median'],
                         'decisiontreeclassifier__max_depth': range(2, 14, 2),
                         'decisiontreeclassifier__min_samples_leaf': range(3, 12, 2)},
             verbose=1)

In [44]:
search.best_params_

{'columntransformer__pipeline-1__simpleimputer__strategy': 'mean',
 'decisiontreeclassifier__max_depth': 6,
 'decisiontreeclassifier__min_samples_leaf': 5}

In [45]:
search.best_score_

0.9280804079087341

In [46]:
# training accuracy
y_train_pred_2 = search.predict(X_train)

accuracy_score(y_train, y_train_pred_2)

0.9691780821917808

In [ ]:
# y_test_pred_2 = search.predict(X_test)

# accuracy_score(y_test, y_test_pred_2)

In [7]:
url = "https://drive.google.com/file/d/1XPpN4NYdldYF8rKZZ4EVWYAhurPJNMti/view?usp=share_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data = df = pd.read_csv(path)

In [8]:
data

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,0,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,1,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,0,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [10]:
X = data.drop(columns=["Id"])
y = X.pop("Expensive")

In [9]:
X 

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,MSZoning,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,RL,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,RL,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,RL,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,RL,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,RL,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,RL,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,RL,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,RL,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,RL,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [52]:
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))
 
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    OneHotEncoder(handle_unknown="ignore",sparse_output=False)
)

preprocessor = make_column_transformer(
        (numeric_pipe,make_column_selector(dtype_include='number')),
        (categoric_pipe, make_column_selector(dtype_include='object'))
)

pipe = make_pipeline(
    (preprocessor),
    (RandomForestClassifier(random_state=123))
)

In [54]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'randomforestclassifier__max_depth':range(15,25),
    'randomforestclassifier__min_samples_leaf':[2,4,6],
    'randomforestclassifier__n_estimators':range(5,15)
}

grid_search = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=5,
    verbose=1,
)

In [55]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DF042520>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                                         (...
                                                                                                        sparse_output=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DF042130>)])),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(random_state=123))]),
             param_grid={'randomforestclassifier__max_depth': range(15, 25),
                         'randomforestclassifier__min_samples_leaf': [2, 4, 6],
                         'randomforestclassifier__n_estimators': range(5, 15)},
             verbose=1)

In [56]:
grid_search.best_estimator_.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DF2458B0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N_A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D7DF2452E0>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=17, min_samples_leaf=2,
                                        n_estimators=11, random_state=123))])

In [5]:
url = "https://drive.google.com/file/d/15PfmTxmavQCT-f7iY9tgwWxm9t4GRees/view"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data_new = pd.read_csv(path)

In [6]:
data_new

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,MSZoning,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,11622,80.0,882.0,2,0,0,1.0,140,120,RH,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1,14267,81.0,1329.0,3,0,0,1.0,393,0,RL,...,Attchd,Unf,TA,TA,Y,NaN,NaN,Gar2,WD,Normal
2,13830,74.0,928.0,3,1,0,2.0,212,0,RL,...,Attchd,Fin,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
3,9978,78.0,926.0,3,1,0,2.0,360,0,RL,...,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,WD,Normal
4,5005,43.0,1280.0,2,0,0,2.0,0,144,RL,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1936,21.0,546.0,3,0,0,0.0,0,0,RM,...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,WD,Normal
1455,1894,21.0,546.0,3,0,0,1.0,0,0,RM,...,CarPort,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1456,20000,160.0,1224.0,4,1,0,2.0,474,0,RL,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1457,10441,62.0,912.0,3,0,0,0.0,80,0,RL,...,NaN,NaN,NaN,NaN,Y,NaN,MnPrv,Shed,WD,Normal


In [58]:
id_column = data_new.pop('Id')

In [59]:
predictions = grid_search.best_estimator_.predict(data_new)

In [60]:
results = pd.DataFrame({'Id':id_column,'Expensive':predictions})

In [66]:
from sklearn import set_config

set_config(transform_output="pandas")

In [69]:
results.to_csv('results.csv',index=False)